In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd /content/drive/My\ Drive/

/content/drive/My Drive


In [3]:

import sys
import numpy as np
import re
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import SimpleRNN
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import os
from nltk.tokenize import TweetTokenizer

from keras import backend as K
import imp
tknzr = TweetTokenizer()


Using TensorFlow backend.


In [4]:
train=[]
test=[]
with open('train1.txt', 'r') as lis:
    for i in lis:
        j = i[:-1]
        train.append(j)
lis.close()
with open('test1.txt', 'r') as lis:
    for i in lis:
        j = i[:-1]
        test.append(j)
lis.close()

nTrain=[]
nTest=[]
for i in train:
    nTrain.extend(tknzr.tokenize(i))
for i in test:
    nTest.extend(tknzr.tokenize(i))
    
raw_text = nTrain
# create mapping of unique chars to integers, and a reverse mapping
chars=set(raw_text)
print(len(chars))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 5
datax = []
datay = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	datax.append([char_to_int[char] for char in seq_in])
	datay.append(char_to_int[seq_out])

print(len(datax[0]),len(datax),len(datay))

dataX=datax[0:100000]
dataY=datay[0:100000]
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
def generator(X_data, y_data, batch_size):
  samples_per_epoch = 100000
  number_of_batches = samples_per_epoch/batch_size
  counter=0

  while 1:
    X_batch = X_data[batch_size*counter:batch_size*(counter+1)]
    y_batch = y_data[batch_size*counter:batch_size*(counter+1)]
    counter += 1
    yield X_batch,y_batch
    #restart counter to yeild data in the next epoch as well
    if counter >= number_of_batches:
        counter = 0
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
y.shape

16090
Total Characters:  793174
Total Vocab:  16090
5 793169 793169
Total Patterns:  100000


(100000, 16090)

In [0]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weight2.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit_generator(generator(X, y, batch_size = 64),steps_per_epoch = len(X) //64,
 epochs = 50,callbacks=callbacks_list)

Epoch 1/50
1562/1562 [==============================] - 52s 34ms/step - loss: 6.2992

Epoch 00001: loss improved from inf to 6.29921, saving model to weight2.hdf5
Epoch 2/50
1562/1562 [==============================] - 50s 32ms/step - loss: 6.0499

Epoch 00002: loss improved from 6.29921 to 6.04951, saving model to weight2.hdf5
Epoch 3/50
1562/1562 [==============================] - 51s 32ms/step - loss: 6.0040

Epoch 00003: loss improved from 6.04951 to 6.00368, saving model to weight2.hdf5
Epoch 4/50
1562/1562 [==============================] - 50s 32ms/step - loss: 6.0067

Epoch 00004: loss did not improve from 6.00368
Epoch 5/50
1562/1562 [==============================] - 50s 32ms/step - loss: 5.8302

Epoch 00005: loss improved from 6.00368 to 5.83001, saving model to weight2.hdf5
Epoch 6/50
1562/1562 [==============================] - 51s 32ms/step - loss: 5.6225

Epoch 00006: loss improved from 5.83001 to 5.62229, saving model to weight2.hdf5
Epoch 7/50
1562/1562 [==============

In [8]:

# load the network weights

filename = "weight2.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print(start)
print(pattern)

pre=" "
for i in range(100):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	pre=pre+result+" "
	seq_in = [int_to_char[value] for value in pattern]
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print(pre)
print("\nDone.")

41137
[10615, 2324, 10867, 7426, 11559]
 , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and , and 

Done.
